In [1]:
import numpy as np

In [6]:
a = np.load("../data/features/cat_features.npz")

In [12]:
for i in range(a["features"].shape[1]):
    print(len(np.unique(a["features"][:, i])))

22
862
5


In [1]:
import torch_directml

In [31]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from src.data_loader import load_features
from src.trainers import train_on_split

import pandas as pd
import numpy as np
from src.model_utils import load_model_bundle
from src.data_loader import load_split, load_features, load_target
from src.config import MODELS_DIR, TARGETS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from pathlib import Path
import sys
import joblib
from sklearn.metrics import classification_report
from src.config import DEFAULT_FEATURES


In [9]:
tfidfn = np.load("../data/features/question_tfidf.npz", allow_pickle=True)

In [22]:
tfidfn['features'].item().shape

(918476, 8000)

In [33]:
features = DEFAULT_FEATURES
target_col = TARGETS["subcategory"]

train_split = load_split("train")
val_split = load_split("val")

x_cat = load_features(['cat_features.npz'], output_format="dense")[train_split]
x_time = load_features(['time_features.npz'], output_format="dense")[train_split]
x_tfidf = load_features(['question_tfidf.npz'], output_format="dense")[train_split]


IndexError: index 123332 is out of bounds for axis 0 with size 1